In [248]:

import boto3
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()
bucket='hack4med'

# wszystkie nasze pliki
przyjecie_file = 'CRACoV-PRZYJECIE.csv'
biochemia_file = 'CRACoV-BIOCHEMIA_3.csv'
echo_file = 'CRACoV-ECHO.csv'
nefro_file = '_CRACoV-NEFRO.csv'
mapowanie_tomografii_file = 'CRACoV-MAPOWANIE-TOMOGRAFII.csv'
radio_file = 'CRACoV-RADIO.csv'
etykiety_file = 'CRACoV-ETYKIETY.csv'

def load_csv(file, skiprows=0):
    data_location = 's3://{}/{}'.format(bucket, file)
    return pd.read_csv(data_location, skiprows=skiprows)

# tutaj mamy wszystkie wczytane dane
przyjecia = load_csv(przyjecie_file)
biochemia = load_csv(biochemia_file)
echo = load_csv(echo_file)
# nefro ma zwalony pierwszy wiersz
nefro = load_csv(nefro_file, 1)
mapowanie_tomografii = load_csv(mapowanie_tomografii_file)
radio = load_csv(radio_file)
etykiety = load_csv(etykiety_file)

etykiety

,LP.,ID_LAB,ZGON_SZPITALNY,PROGRESJA,OIT,ZGON_LUB_OIT
0,1,0,1,1,0,1
1,2,0,0,0,0,0
2,3,0,1,0,0,1
3,4,0,1,0,1,1
4,5,0,1,0,1,1
...,...,...,...,...,...,...
495,495,489,0,0,0,0
496,496,490,0,0,0,0
497,497,491,0,0,1,1
498,498,492,0,0,0,0


In [249]:
biochemia = biochemia.astype({"LP.": int, "ID_LAB": int})
przyjecia = przyjecia.astype({"LP.": int, "ID_LAB": int})

biochemia["ID"] = biochemia["LP."] + 100000*biochemia["ID_LAB"]
przyjecia["ID"] = przyjecia["LP."] + 100000*przyjecia["ID_LAB"]

przyjecia = przyjecia.filter(items=['ID', 'PLEC', 'WIEK', 'WZROST', 'PRZENIESIENIE', 'HOSP_PRZYCZ', 'MASA_CIALA', 'BMI', 'RRS', 'RRD', 'PO2_ATM', 'ODDECH', 'AS', 'NT', 'DM', 'ASTMA', 'POCHP', 'HF', 'AF', 'UDAR', 'CHD', 'MI', 'ZAP_PLUC', 'PCHN', 'DEKSAMETEZON', 'HDCZ', 'BB', 'STATYNA', 'ASA', 'NOAC', 'MRA', 'ACE', 'SARTANY', 'CA_BLOKER'])


In [287]:
import pandas as pd

# biochemia["ID"]
# przyjecia["ID"]


baseCalcData = przyjecia.copy()
# for badanie in ['N58.11.11342_PCT', 'I81.11.1112_CRP', 'G49.122.1113_DD', 'M05_IL-6', 'O59_TNHS', 'N11.126.20.1CITO_MLECZ', 'M37.11.191_KREA', 'C55.103.02_WBC', 'C55.103.02_PLT']:
#     biochemiaCopy = biochemia.copy()
#     biochemiaCopy = biochemiaCopy[biochemiaCopy["KOD"] == badanie].rename(columns={'WYNIK': badanie})
# #     .filter(['ID', 'WYNIK'])
#     print("############")
#     print(badanie)
#     print(biochemiaCopy)
#     print(baseCalcData)
#     baseCalcData = baseCalcData.merge(biochemiaCopy, on=["ID"])


# przyjecia

baseCalcData = baseCalcData.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)



baseCalcData = baseCalcData.replace('nie', 0)
baseCalcData = baseCalcData.replace('nie wiadomo', 1)
baseCalcData = baseCalcData.replace('tak', 2)

baseCalcData = baseCalcData.replace('covid', 1)
baseCalcData = baseCalcData.replace('inny (współistniejący covid)', 1)
baseCalcData = baseCalcData.fillna(0)
# baseCalcData['HOSP_PRZYCZ'].unique() 
zgonluboit = baseCalcData[]

file = 'baseCalcData_bezBadan.csv'
data_location = 's3://{}/{}'.format(bucket, file)
baseCalcData.to_csv(data_location)


In [112]:
# wywalam LP ze wszystkich, bo niepotrzebne

przyjecia = przyjecia.drop('LP.', 1)
biochemia = biochemia.drop('LP.', 1)
echo = echo.drop('LP.', 1)
nefro = nefro.drop('L.P.', 1)
mapowanie_tomografii = mapowanie_tomografii.drop('LP.', 1)
radio = radio.drop('LP.', 1)
etykiety = etykiety.drop('LP.', 1)

etykiety

,ID_LAB,ZGON_SZPITALNY,PROGRESJA,OIT,ZGON_LUB_OIT
0,0,1,1,0,1
1,0,0,0,0,0
2,0,1,0,0,1
3,0,1,0,1,1
4,0,1,0,1,1
...,...,...,...,...,...
495,489,0,0,0,0
496,490,0,0,0,0
497,491,0,0,1,1
498,492,0,0,0,0


In [68]:
# dane z ID_LAB=0 wydaja mi sie uszkodzone (bo sa jakies duplikaty), wiec je wywalam

przyjecia = przyjecia[przyjecia['ID_LAB'] != 0]
biochemia = biochemia[biochemia['ID_LAB'] != 0]
echo = echo[echo['ID_LAB'] != 0]
nefro = nefro[nefro['ID_LAB'] != 0]
mapowanie_tomografii = mapowanie_tomografii[mapowanie_tomografii['ID_LAB'] != 0]
radio = radio[radio['ID_LAB'] != 0]
etykiety = etykiety[etykiety['ID_LAB'] != 0]

In [68]:
# biochemia ma wiele wierszy dla jednego pacjenta
# zamienię to na jeden wiersz z kolumnami, na podstawie ID_LAB, moze nie walnie :P 

# TODO ogarnac jak to dobrze zamienic - nazwe badania mozna chyba pominac
# nazwa kolumny to będzie (KOD_BAD + KOD_PARAMETRU) a wartość to WYNIK
# moze warto tez dodac PRZEK_NORM (jako osobna kolumna)

# TODO sprawdzic tez, czy tylko biochemia czy cos jeszcze

,ID_LAB,NAZWA_BAD,KOD_BAD,KOD_PARAMETRU,NAZWA_PARAMETRU,WYNIK,JM,ZAKRES_REF,PRZEK_NORMY
82,0.0,Prokalcytonina,N58.11.11342,PCT,Prokalcytonina BRAHMS,1.55,ng/ml,"[< 0,50]",H
155,0.0,Prokalcytonina,N58.11.11342,PCT,Prokalcytonina BRAHMS,0.48,ng/ml,"[< 0,50]",NaN
220,0.0,Prokalcytonina,N58.11.11342,PCT,Prokalcytonina,1.75,ng/ml,"[< 0,10]",H
300,0.0,Prokalcytonina,N58.11.11342,PCT,Prokalcytonina,9.25,ng/ml,"[< 0,10]",H
358,0.0,Prokalcytonina,N58.11.11342,PCT,Prokalcytonina BRAHMS,0.08,ng/ml,"[< 0,50]",NaN
...,...,...,...,...,...,...,...,...,...
25563,399.0,Prokalcytonina,N58.11.11342,PCT,Prokalcytonina BRAHMS,0.22,ng/ml,"[< 0,50]",NaN
25642,400.0,Prokalcytonina,N58.11.11342,PCT,Prokalcytonina BRAHMS,0.21,ng/ml,"[< 0,50]",NaN
25716,401.0,Prokalcytonina,N58.11.11342,PCT,Prokalcytonina BRAHMS,0.06,ng/ml,"[< 0,50]",NaN
25779,402.0,Prokalcytonina,N58.11.11342,PCT,Prokalcytonina BRAHMS,0.21,ng/ml,"[< 0,50]",NaN


,ID_LAB,PLEC,WIEK,WZROST,PRZENIESIENIE,HOSP_PRZYCZ,MASA_CIALA,BMI,RRS,RRD,...,DEKSAMETEZON,HDCZ,BB,STATYNA,ASA,NOAC,MRA,ACE,SARTANY,CA_BLOKER
0,0,0,84.0,178.0,Tak,COVID,65.0,20.51,98.0,57.0,...,NaN,Nie,Tak,Tak,Nie,Nie,Nie,Tak,Nie,Nie
1,0,0,83.0,170.0,Nie,INNY (współistniejący COVID),75.0,25.95,122.0,75.0,...,Nie,Nie,Nie,Nie,Nie,Tak,Nie,Nie,Nie,Tak
2,0,0,78.0,178.0,Nie,COVID,90.0,28.40,119.0,65.0,...,NaN,Nie,Nie,Nie,Nie,Nie,Nie,Tak,Nie,Nie
3,0,0,76.0,178.0,Nie,INNY (współistniejący COVID),75.0,23.67,145.0,90.0,...,NaN,Nie,Nie wiadomo,Nie wiadomo,Nie,Nie,Nie wiadomo,Nie wiadomo,Nie wiadomo,Nie wiadomo
4,0,1,72.0,163.0,Nie,COVID,77.0,28.98,131.0,98.0,...,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Tak,Nie,Nie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,490,1,65.0,168.0,Nie,COVID,102.0,36.13,150.0,84.0,...,NaN,Nie,Tak,Tak,Nie,Nie,Nie,Nie,Nie,Nie
496,491,0,77.0,171.0,Nie,COVID,86.0,29.41,149.0,91.0,...,Nie,Tak,Tak,Tak,Tak,Nie,Nie,Nie,Tak,Nie
497,492,0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,493,1,66.0,167.0,Nie,COVID,93.2,33.41,130.0,85.0,...,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Nie,Nie


In [118]:
# laczenie wszystkich tabelek w jedna
# na podstawie ID_LAB

joined = pd.merge(przyjecia, biochemia, on=["ID_LAB"])
joined = pd.merge(joined, echo, on=["ID_LAB"])
joined = pd.merge(joined, nefro, on=["ID_LAB"])
joined = pd.merge(joined, mapowanie_tomografii, on=["ID_LAB"])
joined = pd.merge(joined, radio, on=["ID_LAB"])
joined = pd.merge(joined, etykiety, on=["ID_LAB"])

joined

,ID_LAB,PLEC,WIEK,PRZENIESIENIE,HOSP_PRZYCZ,HOSP_PRZYCZ_INNE,WYKSZTACENIE,MIEJSCE_ZAM,WLK_MIEJSC,AKT_ZAWOD,...,Volume of ground glass in both lungs (cm3),Consolidation volume of both lungs (cm3),Proportion of lung lesions (%),Proportion of ground glass in both lungs (%),Consolidation of both lungs accounted for (%),Pneumonia grade,ZGON_SZPITALNY,PROGRESJA,OIT,ZGON_LUB_OIT
0,11,1,53.0,Nie,COVID,NaN,zawodowe,dom/mieszkanie,miasto > 500,Tak,...,218.66,150.72,14.68,8.80,6.06,critical,0,0,0,0
1,11,1,53.0,Nie,COVID,NaN,zawodowe,dom/mieszkanie,miasto > 500,Tak,...,218.66,150.72,14.68,8.80,6.06,critical,0,0,0,0
2,11,1,53.0,Nie,COVID,NaN,zawodowe,dom/mieszkanie,miasto > 500,Tak,...,218.66,150.72,14.68,8.80,6.06,critical,0,0,0,0
3,11,1,53.0,Nie,COVID,NaN,zawodowe,dom/mieszkanie,miasto > 500,Tak,...,218.66,150.72,14.68,8.80,6.06,critical,0,0,0,0
4,11,1,53.0,Nie,COVID,NaN,zawodowe,dom/mieszkanie,miasto > 500,Tak,...,218.66,150.72,14.68,8.80,6.06,critical,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3996,328,0,45.0,Nie,COVID,NaN,zawodowe,dom/mieszkanie,wieś,Tak,...,231.97,23.75,5.43,4.97,0.51,moderate,0,0,0,0
3997,328,0,45.0,Nie,COVID,NaN,zawodowe,dom/mieszkanie,wieś,Tak,...,231.97,23.75,5.43,4.97,0.51,moderate,0,0,0,0
3998,328,0,45.0,Nie,COVID,NaN,zawodowe,dom/mieszkanie,wieś,Tak,...,231.97,23.75,5.43,4.97,0.51,moderate,0,0,0,0
3999,328,0,45.0,Nie,COVID,NaN,zawodowe,dom/mieszkanie,wieś,Tak,...,231.97,23.75,5.43,4.97,0.51,moderate,0,0,0,0


In [73]:
# tu jakies czyszczenie danych
# wywalanie niepotrzebnych kolumn
# zamiana stringow na liczby (a moze tego sie nie oplaca?)

przyjecia = przyjecia[przyjecia['WIEK'].notna()]
przyjecia['PRZENIESIENIE'] = np.where(przyjecia['PRZENIESIENIE']=='Tak', 1, 0) # 1 - przeniesienie, 0 - inne
# przyjecia['HOSP_PRZYCZ'] = np.where(przyjecia['HOSP_PRZYCZ']=='COVID', 0, np.where(przyjecia['HOSP_PRZYCZ']=='INNY (współistniejący COVID)', 1, 2)) # 0 - COVID, 1 - Współistniejące, 2 - inne

# TODO dokonczyc czyszczenie
# ustalic z Gabrysiem co jest do wywalenia

przyjecia['MIEJSCE_ZAM'].unique() 

array(['dom/mieszkanie', nan, 'DPS'], dtype=object)